In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
import matlab.engine as engi
import matlab as mat
import math
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from scipy.io import loadmat
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sys import stdout
from sklearn.metrics import pairwise_kernels
from sklearn.preprocessing import MinMaxScaler

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path
import matplotlib.pyplot as plt

eng = engi.start_matlab()
eng.addpath(r'matlab_CTKCCA/',nargout=0)

# Get datasets

In [ ]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [ ]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)

    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df.merge(commit_guru_df,on='commit_hash')
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
#     df,cols = apply_cfs(df)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y
def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [ ]:
for project in projects:
    try:
        if project == '.DS_Store':
            continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        X,y = run_self(project)
    except Exception as e:
        print(e)
        continue

# Kernel Two-Sample Test

In [ ]:
def MMD2u(K, m, n):
    """The MMD^2_u unbiased statistic.
    """
    Kx = K[:m, :m]
    Ky = K[m:, m:]
    Kxy = K[:m, m:]
    return 1.0 / (m * (m - 1.0)) * (Kx.sum() - Kx.diagonal().sum()) + \
        1.0 / (n * (n - 1.0)) * (Ky.sum() - Ky.diagonal().sum()) - \
        2.0 / (m * n) * Kxy.sum()


def compute_null_distribution(K, m, n, iterations=10000, verbose=False,
                              random_state=None, marker_interval=1000):
    """Compute the bootstrap null-distribution of MMD2u.
    """
    if type(random_state) == type(np.random.RandomState()):
        rng = random_state
    else:
        rng = np.random.RandomState(random_state)

    mmd2u_null = np.zeros(iterations)
    for i in range(iterations):
        if verbose and (i % marker_interval) == 0:
            print(i),
            stdout.flush()
        idx = rng.permutation(m+n)
        K_i = K[idx, idx[:, None]]
        mmd2u_null[i] = MMD2u(K_i, m, n)

    if verbose:
        print("")

    return mmd2u_null


def compute_null_distribution_given_permutations(K, m, n, permutation,
                                                 iterations=None):
    """Compute the bootstrap null-distribution of MMD2u given
    predefined permutations.

    Note:: verbosity is removed to improve speed.
    """
    if iterations is None:
        iterations = len(permutation)

    mmd2u_null = np.zeros(iterations)
    for i in range(iterations):
        idx = permutation[i]
        K_i = K[idx, idx[:, None]]
        mmd2u_null[i] = MMD2u(K_i, m, n)

    return mmd2u_null


def kernel_two_sample_test(X, Y, kernel_function='rbf', iterations=10000,
                           verbose=False, random_state=None, **kwargs):
    """Compute MMD^2_u, its null distribution and the p-value of the
    kernel two-sample test.

    Note that extra parameters captured by **kwargs will be passed to
    pairwise_kernels() as kernel parameters. E.g. if
    kernel_two_sample_test(..., kernel_function='rbf', gamma=0.1),
    then this will result in getting the kernel through
    kernel_function(metric='rbf', gamma=0.1).
    """
    m = len(X)
    n = len(Y)
    XY = np.vstack([X, Y])
    K = pairwise_kernels(XY, metric=kernel_function, **kwargs)
    mmd2u = MMD2u(K, m, n)
#     if verbose:
#         print("MMD^2_u = %s" % mmd2u)
#         print("Computing the null distribution.")

#     mmd2u_null = compute_null_distribution(K, m, n, iterations,
#                                            verbose=verbose,
#                                            random_state=random_state)
#     p_value = max(1.0/iterations, (mmd2u_null > mmd2u).sum() /
#                   float(iterations))
#     if verbose:
#         print("p-value ~= %s \t (resolution : %s)" % (p_value, 1.0/iterations))
    mmd2u_null, p_value = 0,0
    return mmd2u, mmd2u_null, p_value




   

#     X = np.random.multivariate_normal(mean=muX, cov=sigma2X, size=m)
#     Y = np.random.multivariate_normal(mean=muY, cov=sigma2Y, size=n)
#     print(Y)

#     if d == 2:
#         plt.figure()
#         plt.plot(X[:, 0], X[:, 1], 'bo')
#         plt.plot(Y[:, 0], Y[:, 1], 'rx')

# sigma2 = np.median(pairwise_distances(X, Y, metric='euclidean'))**2
# mmd2u, mmd2u_null, p_value = kernel_two_sample_test(X, Y,
#                                                         kernel_function='rbf',
#                                                         gamma=1.0/sigma2,
#                                                         verbose=True)
#     # mmd2u, mmd2u_null, p_value = kernel_two_sample_test(X, Y,
#     #                                                     kernel_function='linear',
#     #                                                     verbose=True)

#     plt.figure()
#     prob, bins, patches = plt.hist(mmd2u_null, bins=50)
#     plt.plot(mmd2u, prob.max()/30, 'w*', markersize=24, markeredgecolor='k',
#              markeredgewidth=2, label="$MMD^2_u = %s$" % mmd2u)
#     plt.xlabel('$MMD^2_u$')
#     plt.ylabel('$p(MMD^2_u)$')
#     plt.legend(numpoints=1)
#     plt.title('$MMD^2_u$: null-distribution and observed value. $p$-value=%s'
#               % p_value)

# Test all datasets

In [ ]:
results_MMD = {}
results_p = {}
results = []
for s_project in projects:
    try:
        results_MMD[s_project] = {}
        results_p[s_project] = {}
        print(s_project)
        train_X,train_y = load_data(s_project)
        for d_project in projects:
            try:
                test_X,test_y = load_data(d_project)
                X = train_X
                Y = test_X
                sigma2 = np.median(pairwise_distances(X, Y, metric='euclidean'))**2
                mmd2u, mmd2u_null, p_value = kernel_two_sample_test(X, Y,
                                                                kernel_function='rbf',
                                                                gamma=1.0/sigma2,
                                                                verbose=False)
                results_MMD[s_project][d_project] = mmd2u
                results_p[s_project][d_project] = p_value
                results.append([s_project,d_project,mmd2u,p_value])
                print(mmd2u)
            except:
                print("error in :",d_project,test_X.shape)
                continue
    except:
        continue
df_MMD = pd.DataFrame.from_dict(results_MMD,orient='index')
df_p = pd.DataFrame.from_dict(results_p,orient='index')  
df_MMD = df_MMD.reindex(sorted(df_MMD.columns), axis=0)
df_MMD = df_MMD.reindex(sorted(df_MMD.columns), axis=1)
df_MMD.to_csv('result/MMD_100_full.csv')